In [ ]:
from keras import losses, models, optimizers
from keras.models import Sequential
from keras.layers import (Dense, Dropout, Activation, Flatten) 
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.datasets import load_boston 
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from geopy import distance
import tensorflow as tf

In [ ]:
data = pd.read_csv("../input/new-york-city-taxi-fare-prediction/train.csv", sep=',',nrows=7000, parse_dates=["pickup_datetime"])
data

In [ ]:
data = data.dropna(how = 'any', axis = 'rows')

pd.set_option('display.expand_frame_repr', False) 

print((data.pickup_latitude>90).sum())
print((data.pickup_latitude<-90).sum())
print((data.pickup_longitude>90).sum())
print((data.pickup_longitude<-90).sum())

print((data.dropoff_latitude>90).sum())
print((data.dropoff_latitude<-90).sum())
print((data.dropoff_longitude>90).sum())
print((data.dropoff_longitude<-90).sum())

data_n = data.drop(data[(data.pickup_latitude>90) | (data.pickup_latitude<-90) 
| (data.pickup_longitude>90) | (data.pickup_longitude<-90) | (data.dropoff_latitude>90) 
| (data.dropoff_latitude<-90) | (data.dropoff_longitude>90) | (data.dropoff_longitude<-90) 
].index)

data_n = data_n[(data_n.pickup_latitude != 0)]
data_n = data_n[(data_n.pickup_longitude != 0)]
data_n = data_n[(data_n.dropoff_longitude != 0)]
data_n = data_n[(data_n.dropoff_latitude != 0)]

data_n = data_n[(data_n.dropoff_latitude != data_n.pickup_latitude) & (data_n.dropoff_longitude != data_n.pickup_longitude) ]
data_n = data_n.reset_index(drop=True)

print('New size: %d' % len(data_n))

In [ ]:
distance_miles = []

for i in range(len(data_n.pickup_latitude)):
    distance_miles.append(distance.distance((data_n.pickup_latitude[i], data_n.pickup_longitude[i]), (data_n.dropoff_latitude[i], data_n.dropoff_longitude[i])).km)

data_n['distance_miles'] = distance_miles 
data_n

In [ ]:
is_null = pd.isnull(data_n.distance_miles).sum()
print(is_null)

In [ ]:
fig=plt.figure(figsize=(11, 8))
ax1=fig.add_subplot(1,1,1)
ax1.scatter(data_n.distance_miles,data_n.fare_amount,  color='g', alpha=0.2)
ax1.set_title('The graph of distance-related charges')
ax1.set_xlabel("Distance")
ax1.set_ylabel("Payment")

fig3=plt.figure(figsize=(11, 8))
ax5=fig3.add_subplot(1,1,1)
ax5.scatter(data_n.passenger_count, data_n.fare_amount, color='green')
ax5.set_title('The graph of payment depending on the number of passengers')
ax5.set_xlabel('Number of passengers')
ax5.set_ylabel('Payment')

data_n['year'] = data_n.pickup_datetime.apply(lambda t: t.year)
data_n['weekday'] = data_n.pickup_datetime.apply(lambda t: t.weekday())
data_n['hour'] = data_n.pickup_datetime.apply(lambda t: t.hour)

fig2=plt.figure(figsize=(11, 8))
ax5=fig2.add_subplot(1,1,1)
ax5.scatter(data_n.weekday, data_n.fare_amount, color='green')
ax5.set_title('The graph of payment depending on the day of the week')
ax5.set_xlabel('Day of the week')
ax5.set_ylabel('Payment')


fig2=plt.figure(figsize=(11, 8))
ax5=fig2.add_subplot(1,1,1)
ax5.scatter(data_n.hour, data_n.fare_amount, color='green')
ax5.set_title('The graph of payment depending on the call time')
ax5.set_xlabel('Call time')
ax5.set_ylabel('Payment')

In [ ]:
lr = LinearRegression()
x = data_n.distance_miles
y = data_n.fare_amount
x = np.transpose(np.atleast_2d(x))
lr.fit(x,y)
y_pred = lr.predict(x)
mse_lin_rm = mean_squared_error(y, y_pred)
rmse_lin_rm = np.sqrt(mse_lin_rm)
r2_lin_rm = r2_score(y, y_pred) 

print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(mse_lin_rm), np.std(mse_lin_rm) * 2))
print("Mean R^2: %0.2f\n" % (np.mean(r2_lin_rm)))    

fig=plt.figure(figsize=(7, 5))
ax5=fig.add_subplot(1,1,1)
ax5.scatter(data_n.distance_miles,data_n.fare_amount, color='g', alpha=0.2, norm=0.3)
ax5.plot(x,y_pred)
ax5.set_title('Pairwise linear regression by least squares method ')
ax5.set_xlabel('Distance ')
ax5.set_ylabel('Payment $USD')

In [ ]:
# Множинна лінійна регресія
x_mul = data_n[['passenger_count','distance_miles']]
y = data_n.fare_amount

X_train, X_test, y_train, y_test = train_test_split(x_mul, y, 
                                                    test_size=0.2, random_state=5) 
lr_mul = LinearRegression()
lr_mul.fit(X_train, y_train)

p = lr_mul.predict(x_mul)
# Spliting

y_pred = lr_mul.predict(X_test)
mse_lin_mul = mean_squared_error(y_test, y_pred)
r2_mul = r2_score(y_pred, y_test)
 
print("\nAccuracy: %0.2f (+/- %0.2f)" % (np.mean(mse_lin_mul), np.std(mse_lin_mul) * 2))
print("Mean R^2: %0.2f\n" % (np.mean(r2_mul)))        
    
fig=plt.figure(figsize=(7, 5))
mulreg=fig.add_subplot(1,1,1)
mulreg.scatter(p, y, color='r', alpha=0.1)
mulreg.set_title('multiLinear Regression ')

In [ ]:
# Множинна лінійна регресія Lasso

x_mul = data_n[['passenger_count','distance_miles']]
y = data_n.fare_amount

X_train, X_test, y_train, y_test = train_test_split(x_mul, y, test_size=0.1, random_state=0)    

lr_mul = Lasso(alpha=0.1)
lr_mul.fit(X_train, y_train)
y_pred = lr_mul.predict(X_test)
mse_lin_mul = mean_squared_error(y_test, y_pred)
r2_mul = r2_score(y_pred, y_test) 

print("\nAccuracy: %0.2f (+/- %0.2f)" % (np.mean(mse_lin_mul), np.std(mse_lin_mul) * 2))
print("Mean R^2: %0.2f\n" % (np.mean(r2_mul))) 

fig=plt.figure(figsize=(7, 5))
mulreg=fig.add_subplot(1,1,1)
mulreg.scatter(p, y, color='g', alpha=0.1)
mulreg.set_title('Множественная линейная регрессия (Lasso)')

In [ ]:
# Множинна лінійна регресія Ridge

x_mul = data_n[['passenger_count','distance_miles']]
y = data_n.fare_amount

X_train, X_test, y_train, y_test = train_test_split(x_mul, y, test_size=0.24, random_state=0)    

lr_mul = Ridge(alpha=0.1)
lr_mul.fit(X_train, y_train)
y_pred = lr_mul.predict(X_test)
mse_lin_mul = mean_squared_error(y_test, y_pred)
r2_mul = r2_score(y_pred, y_test) 

print("\nAccuracy: %0.2f (+/- %0.2f)" % (np.mean(mse_lin_mul), np.std(mse_lin_mul) * 2))
print("Mean R^2: %0.2f\n" % (np.mean(r2_mul))) 

fig=plt.figure(figsize=(7, 5))
mulreg=fig.add_subplot(1,1,1)
mulreg.scatter(p, y, color='g', alpha=0.1)
mulreg.set_title('Множественная линейная регрессия (Ridge)')

In [ ]:
mlpReg = MLPRegressor(hidden_layer_sizes=(600, ), activation='tanh', 
                      solver='adam', alpha=0.0001, batch_size='auto', 
                      learning_rate='adaptive', learning_rate_init=0.001, 
                      power_t=0.5, max_iter=10000, shuffle=True, random_state=None, 
                      tol=0.0001, verbose=False, warm_start=False, momentum=0.9, 
                      nesterovs_momentum=True, early_stopping=False, 
                      validation_fraction=0.1, beta_1=0.9, beta_2=0.999, 
                      epsilon=1e-08)

mlpReg = mlpReg.fit(X_train, y_train)
y_pred = mlpReg.predict(X_test)
mse_mlp = mean_squared_error(y_test, y_pred)
r2_mlp = r2_score(y_pred, y_test) 
r2_mlp = mlpReg.score(X_test, y_test)

print("\nAccuracy: %0.2f (+/- %0.2f)" % (np.mean(mse_mlp), np.std(mse_mlp) * 2))
print("Mean R^2: %0.2f\n" % (np.mean(r2_mlp))) 
    
fig=plt.figure(figsize=(7, 5))
mulreg=fig.add_subplot(1,1,1)
mulreg.scatter(p, y, color='g', alpha=0.1)
mulreg.set_title('MLP регрессия') 